In [1]:
import os
import numpy as np
import torch as th
import re

/dccstor/cimf/drought_impact/drought/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Combine norm stats
- The stattistcis of the training set might have been computed in several times
- Combine them all here

In [2]:
stats_path = 'pixel_data/nofilter/train'

In [3]:
stat_subset = [os.path.join(stats_path, stat) for stat in os.listdir(stats_path) if stat.endswith('.pt')]

In [109]:
stat_subset

['pixel_data/nofilter/train/norm_stats.pt']

In [110]:
# Load, stack, compute new stats, save
all_bands_min = []
all_bands_max = []

for stat in stat_subset:
    print(stat)
    [all_bands_min_new, all_bands_max_new] = th.load(stat)
    all_bands_min.append(all_bands_min_new)
    all_bands_max.append(all_bands_max_new)
    
new_bands_min = np.stack(all_bands_min).min(axis=0)
new_bands_max = np.stack(all_bands_max).max(axis=0)

pixel_data/nofilter/train/norm_stats.pt


In [114]:
all_bands_max[0][14]

100.0

In [14]:
# Save
th.save([list(new_bands_min), list(new_bands_max)], stats_path+'/norm_stats.pt')

# Check number of files created 

In [2]:
path_data = 'pixel_data/nofilter/test_vaud/x'

In [3]:
data_list = [os.path.join(path_data, f) for f in os.listdir(path_data) if f.endswith('.pt')]

In [4]:
def sort_key(file):
    return int(re.findall(r"\d+", file)[0])

In [5]:
ordered_list = sorted(data_list, key=sort_key)

In [6]:
len(ordered_list)

6000

In [7]:
ordered_list[-1]

'pixel_data/nofilter/test_vaud/x/data_cube_5999.pt'

In [91]:
if 'pixel_data/nofilter/train/x/data_cube_1000.pt' in ordered_list:
    print('ok')

ok


In [116]:
ordered_list[-1]

'pixel_data/arch_exp/val/x/data_cube_4999.pt'

In [5]:
stats_path = 'pixel_data/arch_exp/train/persistance_r2.pt'

In [3]:
mse = th.load(stats_path)

In [4]:
mse

0.011043884350258932

In [6]:
r2 = th.load(stats_path)

In [7]:
r2

-0.0603094978121083

# Get cloud free data

In [ ]:
import torch as th
import pickle
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
from utils.utils_pixel import *

exp = 'nofilter'
batch = 0
batch_size = 40 
epoch = 0

In [3]:
path_data = 'pixel_data/nofilter/train/x'

In [4]:
cloud_free_samples = list()

for batch in range(1000):
    if not batch%50:
        print("at batch", batch)
    img, label = load_batch(batch_size = batch_size, batch_nbr = batch, sample_type = 'pixel_data', split='train', exp='nofilter', n_timesteps_out=1)

    cp_idx = 14
    cp_data = th.cat([img[:,:,cp_idx,:,:] ,label[:,:,cp_idx,:,:]], axis=1)
    
    # Check if sample(s) in cp_data  are fully cloud free
    for sample_id in range(cp_data.size(0)):
        sample = cp_data[sample_id,:,:,:]
        condition = th.all(sample < 0.5)
        if condition:
            cloud_free_samples.append(batch*40+sample_id)

at batch 0


KeyboardInterrupt: 

In [31]:
# Check where the first and last didnt have to get dropped

exp = 'nofilter'
batch = 0
batch_size = 40 
epoch = 0
split = 'val'

path_data = f'pixel_data/{exp}/{split}/x'

cloud_free_samples = list()

for batch in range(100):
    if not batch%50:
        print("at batch", batch)
    img, label = load_batch(batch_size = batch_size, batch_nbr = batch, sample_type = 'pixel_data', split=split, exp=exp, n_timesteps_out=1)

    cp_idx = 14
    cp_data = th.cat([img[:,:,cp_idx,:,:] ,label[:,:,cp_idx,:,:]], axis=1)
    
    # Check if start/end got replaced
    for sample_id in range(cp_data.size(0)):
        first = cp_data[sample_id,0,:,:]
        last = cp_data[sample_id,-1,:,:]
        if first==0 and last==0:
            cloud_free_samples.append(batch*40+sample_id)

at batch 0
at batch 50


In [9]:
# Check where the last point (label) didnt have to get dropped
import torch as th
import pickle
import numpy as np
import os
from utils.utils_pixel import *

exp = 'nofilter'
batch = 0
batch_size = 40 
epoch = 0
split = 'test'

path_data = f'pixel_data/{exp}/{split}/x'

cloud_free_samples = list()

cp_idx = 14
b2_idx = 6
    
for batch in range(100):
    if not batch%50:
        print("at batch", batch)
    img, label = load_batch(batch_size = batch_size, batch_nbr = batch, sample_type = 'pixel_data', split=split, exp=exp, n_timesteps_out=1)

        
    for sample_id in range(cp_data.size(0)):
        cp_data = th.cat([img[sample_id,:,cp_idx,:,:] ,label[sample_id,:,cp_idx,:,:]], axis=0)
        b2_data = th.cat([img[sample_id,:,b2_idx,:,:] ,label[sample_id,:,b2_idx,:,:]], axis=0)
        
        # Check if start/end got replaced
        to_rep = ((b2_data>0.1) & (cp_data>0.01))
        if to_rep.sum() < 9: 
            to_rep = ((b2_data>0.15) & (cp_data>0.01))
        
        if to_rep[-1]:
            cloud_free_samples.append(batch*40+sample_id)

at batch 0
at batch 50


In [10]:
len(cloud_free_samples)

524

# Combine called samples

In [6]:
called_path = 'pixel_data/nofilter/train'
called_subset = [os.path.join(called_path, called) for called in os.listdir(called_path) if 'called_samples' in called]

In [7]:
called_subset

['pixel_data/nofilter/train/called_samples_jura_config_rec_2018_nofilter_p1_1000.pt',
 'pixel_data/nofilter/train/called_samples_jura_config_rec_2018_nofilter_p1_0.pt',
 'pixel_data/nofilter/train/called_samples_jura_config_rec_2018_nofilter_p1_6000.pt',
 'pixel_data/nofilter/train/called_samples_jura_config_rec_2018_nofilter_p1_7000.pt',
 'pixel_data/nofilter/train/called_samples_jura_config_rec_2019_nofilter_p2_0.pt',
 'pixel_data/nofilter/train/called_samples_jura_config_rec_2019_nofilter_p1_3000.pt',
 'pixel_data/nofilter/train/called_samples_jura_config_rec_2019_nofilter_p1_8000.pt',
 'pixel_data/nofilter/train/called_samples_jura_config_rec_2018_nofilter_p1_2000.pt',
 'pixel_data/nofilter/train/called_samples_jura_config_rec_2019_nofilter_p2_2000.pt',
 'pixel_data/nofilter/train/called_samples_jura_config_rec_2019_nofilter_p2_7000.pt',
 'pixel_data/nofilter/train/called_samples_jura_config_rec_2019_nofilter_p1_0.pt',
 'pixel_data/nofilter/train/called_samples_jura_config_rec_2018

In [8]:
len(called_subset)

39

In [23]:
# Load, stack, compute new stats, save
list_called = list()

for called in called_subset:
    list_subset = th.load(called)
    list_called += list_subset
    
# Careful, index for timeseries is different depedning on year and p1 vs p2

In [24]:
# Check if same pixel called multiple times
list_pixs = [x[0][1:] for x in list_called]

In [25]:
len(list_pixs)

39000

In [26]:
len(set(list_pixs))

39000